### Please install the required Python modules/SDKs

In [ ]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

## Please use this Jupyter notebook as solution to your exercise

### Import utility functions and Python modules 

In [ ]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import os, time, uuid

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

In [ ]:
# helper function
def show_image_in_cell(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(20,10))
    plt.imshow(img)
    plt.show()

### TODO: Add Your Training Endpoint Resource

In [ ]:
TRAINING_ENDPOINT = "ENTER TRAINING ENDPOINT HERE"
training_key = "ENTER TRAINING RESOURCE KEY HERE"
training_resource_id = 'ENTER TRAINING RESOURCE ID HERE'

In [ ]:
PREDICTION_ENDPOINT = 'ENTER PREDICTION RESOURCE KEY HERE'
prediction_key = "ENTER PREDICTION RESOURCE KEY HERE"
prediction_resource_id = "ENTER PREDICTION RESOURCE ID HERE"

In [ ]:
# Instantiate and authenticate the training client with endpoint and key
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)
trainer.api_version

### TODO: Creating Training Project

In [ ]:
# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Todo: create a new project
print ("Your Object Detection Training project has been created. Please move on.")
project_name = 'CHANGE THIS'
project = None

In [ ]:
# Getting Project Details as collective information
project.as_dict()

In [ ]:
project.status

### TODO: Adding Tags 

In [ ]:
# Todo: add tags based on training requirements 
tag_1 = None
tag_2 = None

## VERY IMPORTANT - PAUSE HERE
### Now, please go to the Custom Vision portal, upload and label your training images

### Once you have uploaded and labeled all the training images at the portal, you can come back to this notebook and start the training process. 

### TODO: Start the Object Detection Training

In [ ]:
# Todo: start the training
iteration = None

# We will keep checking every 10 seconds during the training progress
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

### TODO: Check Model Performance After Training

In [ ]:
iteration.as_dict()

In [ ]:
iteration_list = trainer.get_iterations(project.id)
for iteration_item in iteration_list:
    print(iteration_item)

In [ ]:
# Todo: check the preformance
model_perf = None

In [ ]:
model_perf.as_dict()

### TODO: Improve Your Model Performance
- Evaluate model performance by looking at the precision and recall values using the Python SDK.
- Add more images via the Custom Vision portal and perform more training iterations to improve your model performance using the Python SDK.

### TODO: Publishing the Model to the Project Endpoint

In [ ]:
# Todo: set the Iteration Name.
publish_iteration_name = "CHANGE THIS"

# Todo: publish it to the project endpoint
trainer.publish_iteration()
print ("Done!")

### TODO: Performing Prediction

In [ ]:
# Todo: set the right local path
local_image_path = 'CHANGE THIS'

In [ ]:
# Todo: define a perform_prediction function
def perform_prediction(image_file_name):
    with open(os.path.join (local_image_path,  image_file_name), "rb") as image_contents:
       # Todo: set predict object for object detection
        results = None
        # Display the results.
        for prediction in results.predictions:
            print("\t" + prediction.tag_name +
                  ": {0:.2f}%".format(prediction.probability * 100))

In [ ]:
# Todo: upload a test image and set the name of your own image
file_name = 'CHANGE THIS'

In [ ]:
perform_prediction(file_name)

In [ ]:
# Checking the Image
with open(os.path.join (local_image_path, file_name), 'rb') as img_code:
    img_view_ready = Image.open(img_code)
    plt.figure()
    plt.imshow(img_view_ready)

In [ ]:
# Todo: upload another test image and set the name of your own image
file_name_2 = 'CHANGE THIS'

In [ ]:
# Checking the Image
with open(os.path.join (local_image_path, file_name_2), 'rb') as img_code:
    img_view_ready = Image.open(img_code)
    plt.figure()
    plt.imshow(img_view_ready)

### TODO: Exporting Model 
Note: This step may take a long time. And make sure the iteration is exportable.

In [ ]:
# Todo: set the platform and flavor
platform = "CHANGE THIS"
flavor = "CHANGE THIS"

In [ ]:
# Todo: set the export_iteration method
export_process = None

In [ ]:
print(export_process.output)

In [ ]:
print(export_process.output.status)

In [ ]:
# Code snippet is from Azure SDK and Documentation
# https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/export-programmatically
# This step may take long time 
while (export_process.output.status == "Exporting"):
    print ("Waiting 10 seconds...")
    time.sleep(10)
    exports = trainer.get_exports(project_id, selected_iteration_id)
    for e in exports:
        if e.platform == export_process.output.platform and e.flavor == export_process.output.flavor:
            export = e
            break
    print("Export status is: ", export_process.output.status)

In [ ]:
print(export_process.output.status)

In [ ]:
print(export_process.output.download_uri)

In [ ]:
# Downloading the model from url.
if export_process.output.status == "Done":
    # Ready to Download
    model_export_file = requests.get(export_process.output.download_uri)
    # You can set the name of the download file here
    with open("model.zip", "wb") as file:
        file.write(model_export_file.content)